In [19]:
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
import wandb
DOWNLOAD=False

In [20]:
if DOWNLOAD:
    api = wandb.Api(timeout=30)
    runs = api.runs("eth-ticinesi/FORL_Briscola",
                    filters={"$and" : [{"tags": {"$in": ["P2"]}}, {"tags": {"$nin": ["COM"]}}]})


    for r in runs:
        print(r.name)
        keys = ["test/reward_model_bad_vs_random_mean", "test/reward_model_good_vs_random_mean"]
        tmp = r.scan_history(keys=keys)

        arch = 'nn' if 'nn' in r.tags else 'rnn'

        acc = []
        for i,t in enumerate(tmp):
            d = pd.DataFrame(t, index=[0])
            d['step'] = i*(1000*16) #16'000 games
            d['arch'] = arch
            acc.append(d)
        df = pd.concat(acc)
        df.to_csv(f'./P2_nocom_good_bad/{r.name}_{r.id}.csv', index=False)

In [21]:
if DOWNLOAD:
    api = wandb.Api(timeout=30)
    runs = api.runs("eth-ticinesi/FORL_Briscola",
                    filters={"$and" : [{"tags": {"$in": ["P1"]}}, {"tags": {"$nin": ["COM"]}}]})

    import os
    for r in runs:
        if os.path.isfile(f'./P1_nocom_good_bad/{r.name}.csv'):
            continue
        print(r.name)
        keys = ["test/reward_model_bad_vs_random_mean", "test/reward_model_good_vs_random_mean"]
        tmp = r.scan_history(keys=keys)

        arch = 'nn' if 'nn' in r.tags else 'rnn'

        acc = []
        for i,t in enumerate(tmp):
            d = pd.DataFrame(t, index=[0])
            d['step'] = i*(1000*16) #16'000 games
            d['arch'] = arch
            acc.append(d)
        df = pd.concat(acc)
        df.to_csv(f'./P1_nocom_good_bad/{r.name}.csv', index=False)

In [22]:
def save_fig(fig, name):
    fig.update_layout(legend_title_text='', showlegend=True)
    fig.update_layout(legend=dict(
        orientation="h",
        yanchor="bottom",
        y=1.02,
        xanchor="right",
        x=1
    ))

    fig.update_layout(width =500, height=220, 
                    font_family="Serif", font_size=11, title_font_size=13, 
                    margin_l=5, margin_t=1, margin_b=1, margin_r=5)

    fig.update_layout(title="")
    fig.update_yaxes(nticks=12)
    import plotly.io as pio
    #save a figure of 300dpi, width 1.5 inches, height 0.75inches
    pio.write_image(fig, f"./img/{name}.png")
    fig.show()

In [23]:
df128_part_2 = pd.read_csv('./P2_nocom_good_bad/RNN h=128 m=128 P2_1onnesoo.csv')
df128_part_1 = pd.read_csv('./P2_nocom_good_bad/RNN h=128 m=128 P2_95o6x1dd.csv')

df128_part_2.drop(0, axis=0)
df128_part_2['step'] += 6400000 - 16000

df128 = pd.concat([df128_part_1.iloc[:401], df128_part_2])
df128.to_csv('./P2_nocom_good_bad/RNN h=128 m=128 P2.csv', index=False)

In [24]:
names_nn_P2 = [
 'NN h=32 sample_batch=False P2',
 'NN h=32 sample_batch=True P2',
 'NN h=64 sample_batch=True P2',
 'NN h=64 sample_batch=False P2',
 ]

names_rnn_P2 = ['RNN h=128 m=128 P2',
 'RNN h=128 m=64 P2',
 'RNN h=64 m=128 P2',
 'RNN h=64 m=64 P2',
 'RNN h=32 m=128 P2',
 'RNN h=32 m=64 P2']

names_nn_P1 = [
 'NN h=32 sample_batch=False P1',
 'NN h=32 sample_batch=True P1',
 'NN h=64 sample_batch=True P1',
 'NN h=64 sample_batch=False P1',
 ]

names_rnn_P1 = [
 'RNN h=128 m=64 P1',
 'RNN h=64 m=128 P1',
 'RNN h=64 m=64 P1',
 'RNN h=32 m=128 P1',
 'RNN h=32 m=64 P1']

In [25]:
def plot_good_bad(display_arch, names_list, file_name, source_dir):
    fig_bad = go.Figure()
    fig_good = go.Figure()
    fig_mean = go.Figure()
    for name in names_list:
        df = pd.read_csv(f'./{source_dir}/{name}.csv')
        display_name = name
        if display_arch  == 'MLP':
            display_name = display_name.replace('NN', 'MLP')
            display_name = display_name.replace('sample_batch', 'sb')
        fig_bad.add_trace(go.Scatter(x=df['step'], y=df["test/reward_model_bad_vs_random_mean"].ewm(halflife=5).mean(), name=display_name))
        fig_good.add_trace(go.Scatter(x=df['step'], y=df["test/reward_model_good_vs_random_mean"].ewm(halflife=5).mean(), name=display_name))
        fig_mean.add_trace(go.Scatter(x=df['step'], y=df[["test/reward_model_bad_vs_random_mean", "test/reward_model_good_vs_random_mean"]].mean(axis=1).ewm(halflife=5).mean(), name=display_name))
    fig_bad.update_layout(title=f'{display_arch} Caller & Callee vs random', xaxis_title='Number of games', yaxis_title='Points')
    fig_bad.add_hline(y=89.711, line_dash="dot", line_color="green", annotation_text='heuristic', annotation_position='top left')
    fig_bad.add_hline(y=78.256, line_dash="dot", line_color="gray", annotation_text='random')

    fig_good.update_layout(title=f'{display_arch} Good vs random', xaxis_title='Number of games', yaxis_title='Points')
    fig_good.add_hline(y=46.973, line_dash="dot", line_color="green", annotation_text='heuristic', annotation_position='top left')
    fig_good.add_hline(y=41.744, line_dash="dot", line_color="gray", annotation_text='random')

    fig_mean.update_layout(title=f'{display_arch} Model vs random', xaxis_title='Number of games', yaxis_title='Points')
    fig_mean.add_hline(y=68.342, line_dash="dot", line_color="green", annotation_text='heuristic', annotation_position='top left')
    fig_mean.add_hline(y=60, line_dash="dot", line_color="gray", annotation_text='random')
    fig_bad.show()
    save_fig(fig_bad, f'{file_name}_bad')
    fig_good.show()
    save_fig(fig_good, f'{file_name}_good')
    fig_mean.show()
    save_fig(fig_mean, f'{file_name}_mean')


In [26]:

plot_good_bad('MLP', names_nn_P2, 'P2_mlp', source_dir='P2_nocom_good_bad')
plot_good_bad('RNN', names_rnn_P2, 'P2_rnn', source_dir='P2_nocom_good_bad')

In [27]:

plot_good_bad('MLP', names_nn_P1, 'P1_mlp', source_dir='P1_nocom_good_bad')
plot_good_bad('RNN', names_rnn_P1, 'P1_rnn', source_dir='P1_nocom_good_bad')